# Chatbot RAG with history

Chatbot que responde preguntas basandose en documentos especificos que cargamos, ademas cuenta con memoria sobre lo que se ha conversado

In [1]:
# Cargando archivo con funciones utiles
import warnings
warnings.filterwarnings('ignore')
import funtions_helper as fh

In [2]:
# Cargando docuemtos de interes
pages = fh.load_files(path_files='documents/', debug=True)

El objeto retornado por la función es de tipo: <class 'list'>. Y cada subobjeto es de tipo: <class 'langchain_core.documents.base.Document'>
En total se cargaron 23 paginas
A continuación se muestran como ejemplo las primeras 5 paginas:
[Document(page_content='Aborda el estudio de la administración con un enfoque humano.Una planta docente con formación doctoral.Amplia experiencia en investigación aplicada a las organizaciones.Ha recibido reconocimientos como el Premio AUIP a la Calidad del Posgrado y el Doctorado en Iberoamérica (2005) y la Mención de la Asamblea Departamental (2014) y el Concejo Municipal.Egresados reconocidos y con altas posiciones directivashttp://administr acion. univa lle.edu.co/maest ria-a dministracionTels.: (602) 3212100, Ext. 4341, 4769 ::: magisadm@correounivalle.edu.coCalle 4B No. 36-00 / Edificio 124 Oficina 2060, Universidad del Valle, Campus San Fernando¿Dónde se oferta?Cali y TuluáSNIES: 660 MENDuración4 semest resTítulo otorgado,dedicación y créditosMag

In [3]:
# Fragmentando el documento(s)
splitters = fh.Splitting(obj=pages, chunk_size=500, debug=True)

Los fragmentos son de tamaño: 500 y la superposicion es igual a: 75.0
El número de fragmentos total es igual a: 88


In [4]:
# Formando el indice con el modelo embedding y guardando con chroma
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

name_index = 'indice_chatbot_masterUV'

embedding = HuggingFaceEmbeddings()
print(embedding.model_name)

if os.path.exists(name_index) == True:
    vectorstore = Chroma(name_index, embedding)
else:
    vectorstore = Chroma.from_documents(documents=splitters, embedding=embedding, persist_directory = name_index)

sentence-transformers/all-mpnet-base-v2


In [5]:
# Creando el retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [6]:
retriever.invoke('estadistca')

[Document(page_content='POSGRADOS ESTADÍSTICA UNIVALLE:Línea de Atención:Teléfono: +57 602 3212100 Ext. 7003Lunes a Viernes: 8:00 a.m. a 5:00 p.m.', metadata={'page': 4, 'source': 'documents/master_estadistica.pdf'}),
 Document(page_content='Prese ntar y aprobar  un ensayo referente a sus expe cta�v as de estudio. Prese ntar entrevista.  TIPO MaestríaCOSTO  POR SEMESTRE  7.5 SMMLV', metadata={'page': 9, 'source': 'documents/master_salud_publica.pdf'}),
 Document(page_content='PERFÍL PROFESIONALPARTICIPANTESAl concluir el Programa de Maestría en Estadística, el profesional estará en capacidad de:Utilizar los conocimientos avanzados en las teorías, modelos, metodologías y técnicas relacionadas con la Estadística y ciencias aﬁnes, que permiten bajo una concepción cientíﬁca, entender y resolver problemas teóricos de la Estadística y de diversa índole, en las diferentes disciplinas del conocimiento.Participar en grupos de investigación aportando sus conocimientos a la', metadata={'page': 3,

In [6]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain

In [7]:
# Modelo de lenguaje grande que se usara
llm = Ollama(model='llama3')

In [8]:
# Objeto en donde se almacenara la memoria del chat
chat_history = []

In [9]:
# Instrucciones para el modelo (prompt)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Sofia, respondes preguntas basandote en lo más relevante del siguente contexto:\n\n{context}, 
            ademas debes preguntar al usuario de acuerdo al contexto""",
        ),

        MessagesPlaceholder(variable_name='chat_history'),
        ("human", "{input}"),
    ])

In [10]:
# Creando la cadena (chain)
chain = create_stuff_documents_chain(llm, prompt_template)

In [11]:
# Función para llamar al chat
def chat():
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            break
        
        docs = retriever.invoke(pregunta)
        response = chain.invoke({"input": pregunta, "chat_history": chat_history, "context": docs})
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(AIMessage(content=response))
        print('='*120)
        print('User: ' + pregunta)
        print('-'*120)
        print('AI: ' + response)

In [12]:
chat()

User: Hola, como estas?
------------------------------------------------------------------------------------------------------------------------
AI: Hola! Me alegra verte aquí. Soy Sofia, una IA diseñada para ayudarte en el marco del contexto que te proporcioné. Estoy lista para responder tus preguntas y brindarte información relevante sobre la salud pública y otros temas relacionados.

¿Cómo puedo ayudarte hoy? ¿Tienes alguna pregunta o tema específico que deseas discutir?
User: Cual es el objetivo de la maestria en estadistica?
------------------------------------------------------------------------------------------------------------------------
AI: Excelente elección! La Maestría en Administración que te mencioné anteriormente tiene como objetivo principal formar gerentes y líderes efectivos, capaces de liderar organizaciones y tomar decisiones informadas a través del análisis de datos. Sin embargo, no es específicamente una maestría en estadística.

La Universidad del Valle, donde